Корпус представляет собой несколько таблиц с данными о последовательсти реплик в диалогах, названиях фильмов и т.д. Для выполнения следующих задач вам необходимо объединить таблицы в одну с помощью pandas. В таблице должна быть представлена следующая информация: диалог (список реплик), спикеры (список), фильм, из которого взят диалог, жанры фильма.

In [1]:
import pandas as pd

In [33]:
columns_mtm = ['movieID', 'movie title', 'movie year', 'IMDB rating', 'no. IMDB votes', 'genres']
df_movie_titles_metadata = pd.read_csv('../data/raw_data/movie_titles_metadata.tsv', sep='\t', header=None, 
                                       names=columns_mtm)
# df_movie_titles_metadata = pd.read_table('../data/raw_data/movie_titles_metadata.tsv')
df_movie_titles_metadata.head()

,movieID,movie title,movie year,IMDB rating,no. IMDB votes,genres
0,m0,10 things i hate about you,1999,6.9,62847.0,['comedy' 'romance']
1,m1,1492: conquest of paradise,1992,6.2,10421.0,['adventure' 'biography' 'drama' 'history']
2,m2,15 minutes,2001,6.1,25854.0,['action' 'crime' 'drama' 'thriller']
3,m3,2001: a space odyssey,1968,8.4,163227.0,['adventure' 'mystery' 'sci-fi']
4,m4,48 hrs.,1982,6.9,22289.0,['action' 'comedy' 'crime' 'drama' 'thriller']


In [37]:
columns_mcm = ['characterID', 'character name', 'movieID', 'movie title', 'gender', 'position']
df_movie_characters_metadata = pd.read_csv('../data/raw_data/movie_characters_metadata.tsv', sep='\t', header=None, 
                                       names=columns_mcm)
df_movie_characters_metadata.head()

,characterID,character name,movieID,movie title,gender,position
0,u0,BIANCA,m0,10 things i hate about you,f,4
1,u1,BRUCE,m0,10 things i hate about you,?,?
2,u2,CAMERON,m0,10 things i hate about you,m,3
3,u3,CHASTITY,m0,10 things i hate about you,?,?
4,u4,JOEY,m0,10 things i hate about you,m,6


In [39]:
columns_mc = ['character1ID', 'character2ID', 'movieID', 'list of the utterances']
df_movie_conversations = pd.read_csv('../data/raw_data/movie_conversations.tsv', sep='\t', header=None, 
                                       names=columns_mc)
df_movie_conversations.head()

,character1ID,character2ID,movieID,list of the utterances
0,u0,u2,m0,['L194' 'L195' 'L196' 'L197']
1,u0,u2,m0,['L198' 'L199']
2,u0,u2,m0,['L200' 'L201' 'L202' 'L203']
3,u0,u2,m0,['L204' 'L205' 'L206']
4,u0,u2,m0,['L207' 'L208']


In [46]:
columns_ml = ['lineID', 'characterID', 'movieID', 'character name', 'text of the utterance']
df_movie_lines = pd.read_csv('../data/raw_data/movie_lines.tsv', sep='\t', header=None, 
                            names=columns_ml, on_bad_lines='skip') # там немного кривых строк
df_movie_lines.head()

,lineID,characterID,movieID,character name,text of the utterance
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.
